In [215]:
import pyspark
import numpy as np
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.getOrCreate()

In [216]:
filepath = "../../dp-test-datasets/data/PUMS_california_demographics/data.csv"
df5 = spark.read.load(filepath, format="csv", sep=","
                    ,inferSchema="true", header="true")


df5 = df5.withColumn("random",rand())
df5 = df5.filter((df5.random<.001))
N = df5.count()
num_partitions = N // 60
df5 = df5.repartitionByRange(num_partitions, "random")
# filter to make the dataframe smaller, for testing


r = 50
df5.persist()


DataFrame[_c0: int, state: int, puma: int, sex: int, age: int, educ: int, income: decimal(6,0), latino: int, black: int, asian: int, married: int, random: double]

In [217]:
dftest = df5.filter((df5.random<.0001))
groupByCols = ["sex","married"]
dftest.count()
dfkeys = df5.select(groupByCols).distinct()

keys_pdf = dfkeys.toPandas()


In [218]:
r  = 50

def wavg(group, avg_name, weight_name):
    d = group[avg_name]
    w = group[weight_name]
    try:
        return pd.Series({'weighted_avg': (d * w).sum() / w.sum()})
    except ZeroDivisionError:
        return pd.Series({'weighted_avg': float('nan')})
        
     
    
def synopsis(partition):
    #convert partition into pandas dataframe
    pList = list(partition)
    pLen = len(pList)
    vals = [None] * pLen   
    for i in range(pLen):
         # fill array with Row      
        vals[i] = pList[i]
    pdf = pd.DataFrame(vals) 
    pdf.columns = vals[0].__fields__
    pdf['income'] = pd.to_numeric(pdf.income, errors='coerce')
   
    # sample multinomial to get array of weights       
    weights = np.random.multinomial(N, [1/pLen]*pLen, size=r)
   
    
    # For each run:
    # For group by key combination (fill empties with 0)
    # run the estimator on the groups and collect the result.     
    all_runs = None 
  
    for i in range(r):   
        pdf['weights'] = weights[i]
        merged = pd.merge(pdf, keys_pdf, how='outer', on= groupByCols)
        merged["weights"] = merged["weights"].fillna(0)
        avg_income = merged.groupby(groupByCols).apply(wavg, "income", "weights").reset_index()
        if (all_runs is None):
            all_runs = avg_income
        else:
            all_runs = all_runs.append(avg_income)
            
    mean_runs = all_runs.groupby(groupByCols).mean().reset_index()
    yield mean_runs
        
result_list = df5.rdd.mapPartitions(synopsis).collect()
all_results = result_list[0]
for i in range(1,len(resultList)):
    all_results = all_results.append(result_list)

all_results.groupby(groupByCols).mean()

weighted_avg
sex married              
0   0        25710.993439
    1        48220.366655
1   0        23518.815482
    1        21568.187154

In [219]:

import pyspark.sql.functions as f
df5.groupBy("sex","married").mean().select("sex","married","avg(income)").orderBy(groupByCols).show()

+---+-------+-----------+
|sex|married|avg(income)|
+---+-------+-----------+
|  0|      0| 25509.5843|
|  0|      1| 47713.7714|
|  1|      0| 24053.6766|
|  1|      1| 21572.5231|
+---+-------+-----------+

